# Задание


В рамках этого заадания мы будем создавать программу, которая получая на вход отзыв, будет предсказывать, является ли отзыв положительным или отрицательным. Делать мы будем это таким образом: мы возьмём некоторое число заранее размеченных как положительные или отрицательные отзывов, выделим те слова, которые встречаются только в положительных или только в отрицательных отзывах, и будем считать, каких слов в поступившем нам на проверку отзыве больше.

Мы будем работать по заранее определённому пайплайну:

Сначала нам надо скачать дату -- соберите как минимум 60 (30 положительных и 30 отрицательных) отзывов на похожие продукты (не надо мешать отзывы на отели с отзывами на ноутбуки) для составления " тонального словаря" (чем больше отзывов, тем лучше) и 10 отзывов для проверки качества. 2 балла в случае сбора путём парсинга, 1 - если найдете уже готовые данные или просто закопипастите без парсинга

Токенизируйте слова, приведите их к нижнему регистру и к начальной форме (1 балл за токенизацию, 1 - за начальную форму)

Составьте 2 множества - в одном будут слова, которые встречаются только в положительных отзывах, а в другом - встречающиеся только в отрицательных. Попробуйте поиграть с частотностями и исключить шум (к примеру, выбросить слова, встречающиеся 1-2 раза) (3 балла) (если у вас получились пустые множества, уберите фильтр по частотности или увеличьте выборку)

Создайте функцию, которая будет определять, положительный ли отзыв или отрицательный в зависимости от того, какие слова встретились в нём, и посчитайте качество при помощи accuracy (1 - за коректно работающую функцию, 1 - за подсчёт accuracy)

Предложите как минимум 2 способа улучшить эту программу с помощью добавления к ней любых мулек - просто словами, писать улучшающий код не надо (1 балл)

Логичность и чистота кода (1 балл)

Да, тут 11 баллов - один можно потерять.

В случае, если после долгих мучений в п. 3 множества по объективным причинам не получается (покажите, что пытались) - отправляйте жабу - зачьтём полный балл

*Импортирую бутифулсуп для парсинга*

In [1]:
import requests
from bs4 import BeautifulSoup


Беру все нужные урлы для парсинга и вытаскиваю из низ тексты в созданные файлы good/bad_reviews.txt 

In [88]:
urls = ['https://www.banki.ru/services/responses/bank/talkbank/?is_countable=on', 
        'https://www.banki.ru/services/responses/bank/talkbank/?is_countable=on&page=2&isMobile=0',
       'https://www.banki.ru/services/responses/bank/talkbank/?is_countable=on&page=3&isMobile=0',
       'https://www.banki.ru/services/responses/bank/talkbank/?is_countable=on&page=4&isMobile=0',
       'https://www.banki.ru/services/responses/bank/talkbank/?is_countable=on&page=5&isMobile=0',
       'https://www.banki.ru/services/responses/bank/talkbank/?is_countable=on&page=6&isMobile=0',
       'https://www.banki.ru/services/responses/bank/talkbank/?is_countable=on&page=7&isMobile=0']

reviews = []
for url in urls:
    r = requests.get(url) 
    soup = BeautifulSoup(r.text) 
    tables=soup.find_all('a', class_='header-h3', href=True)
    
    for t in tables:
        link = t.get('href')
        review_address = 'https://www.banki.ru' + link
        reviews.append(review_address)
    

In [89]:
print(len(reviews), reviews[0])

161 https://www.banki.ru/services/responses/bank/response/10555350/


In [23]:
with open('good_reviews.txt', 'w', encoding='utf-8') as f:
    f.write('')
    
with open('bad_reviews.txt', 'w', encoding='utf-8') as ff:
    ff.write('')

In [16]:
!pip3 install tqdm

     |████████████████████████████████| 76 kB 748 kB/s eta 0:00:01


In [17]:
from tqdm import tqdm

In [24]:
for rev in tqdm(reviews):
    r = requests.get(rev)
    soup = BeautifulSoup(r.text)
    tag = soup.find('div', class_='flexbox flexbox--inline flexbox--row flexbox--gap_xsmall flexbox--align-items_baseline')
    spans = tag.find_all('span')
    tag = spans[1]
    if int(tag.text) > 3:
        filename = 'good_reviews.txt'
    elif int(tag.text) < 3:
        filename = 'bad_reviews.txt'
    text = soup.find('div', class_='article-text response-page__text markup-inside-small markup-inside-small--bullet')
    with open(filename, 'a', encoding='utf-8') as file:
        file.write(text.text)
        file.write('\n=====\n')


100%|█████████████████████████████████████████| 161/161 [01:35<00:00,  1.68it/s]


In [25]:
with open('good_reviews.txt', 'r') as file:
    text = file.read()
    g = text.split('=====')
print(len(g))

117


In [26]:
with open('bad_reviews.txt', 'r') as file:
    text = file.read()
    b = text.split('=====')
print(len(b))

46


In [34]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tarasandrushko/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Привожу текст к нужному формату(нижний регистр, убираю запятые)

In [35]:
good_corpus = " ".join(g)
good_corpus = good_corpus.lower()
good_corpus = nltk.word_tokenize(good_corpus)
print(good_corpus[:100])

['что-то', 'упустил', 'момент', 'с', 'пакетами', 'обслуживания', 'в', 'talkbank', ',', 'видимо', ',', 'сразу', 'как-то', 'не', 'прочитал', 'в', 'тарифах', '.', 'пытался', 'вывести', 'деньги', 'на', 'другую', 'карту', ',', 'но', 'постоянно', 'выдавало', ',', 'что', 'будет', 'списана', 'комиссия', '.', 'мне', 'как-то', 'это', 'не', 'понравилось.написал', 'в', 'поддержку', 'им', 'вечером', ',', 'в', 'часиков', '6', ',', 'думал', ',', 'что', 'ответят', 'только', 'утром', ',', 'но', 'через', '10', 'минут', 'пришел', 'ответ', 'поддержки', '.', 'мне', 'сообщили', 'о', 'расширенном', 'пакете', ',', 'помогли', 'его', 'сменить', 'и', 'подключить', '.', 'сейчас', 'все', 'работает', ',', 'благодарю', 'за', 'помощь', '.', 'оформила', 'самозанятость', 'через', 'talkbank', '.', 'проблем', 'особых', 'не', 'возникло', ',', 'не', 'понравилось', 'только', 'то', ',', 'что', 'моего']


In [40]:
bad_corpus = " ".join(b)
bad_corpus = bad_corpus.lower()
bad_corpus = nltk.word_tokenize(bad_corpus)
print(bad_corpus[:100])

['добрый', 'вечер', ',', 'пришёл', 'и', 'мой', 'черёд', 'удивляться', 'действиям', 'банка.решила', 'я', 'как', 'то', 'проверить', 'баланс', 'по', 'картам', 'данного', 'банка', ',', 'да', 'закинуть', 'остатки', 'финансов', 'себе', 'на', 'телефон', ',', 'на', 'счёт', '.', 'да', 'не', 'тут', 'то', 'было', '!', 'на', 'мой', 'запрос', '«', 'баланс', '»', 'мне', 'пришёл', 'приинтереснейший', 'ответ', '.', ':', 'а', 'у', 'вас', 'нет', 'карт', '!', 'сначала', 'глазам', 'своим', 'не', 'поверила', ',', 'провела', 'повторно', '-', 'та', 'дам', '-', 'ответ', 'тот', 'же', '.', 'думаю', 'интересно', 'конечно', ',', 'куда', 'же', 'делись', 'мои', 'карты', '(', 'их', 'две', 'в', 'данном', 'банке', ')', 'на', 'одной', 'были', 'копейки', 'какие', 'то', ',', 'пару', 'рублей', 'может', ',', 'на', 'другой']


In [38]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [39]:
g_out = ""
for gc in good_corpus:
    p = morph.parse(gc)[0]
    g_out += p.normal_form
    g_out += ' '

g_out[:100]

'что-то упустить момент с пакет обслуживание в talkbank , видимо , сразу как-то не прочитать в тариф '

In [41]:
b_out = ""
for bc in bad_corpus:
    p = morph.parse(bc)[0]
    b_out += p.normal_form
    b_out += ' '

b_out[:100]

'добрый вечер , прийти и мой череда удивляться действие банка.решить я как то проверить баланс по кар'

In [42]:
import string

In [43]:
good_corpus = g_out.translate(str.maketrans('', '', string.punctuation))
bad_corpus = b_out.translate(str.maketrans('', '', string.punctuation))

Создаю частотный словарь и убираю "шум"

In [47]:
import os
import re
from collections import Counter

In [48]:
bad_c = Counter(bad_corpus.split(' '))

In [49]:
bad_c.most_common(20)

[('', 1555),
 ('в', 226),
 ('не', 207),
 ('и', 200),
 ('я', 190),
 ('карта', 143),
 ('на', 138),
 ('с', 128),
 ('что', 104),
 ('быть', 92),
 ('они', 76),
 ('банк', 71),
 ('это', 70),
 ('по', 68),
 ('но', 68),
 ('день', 54),
 ('а', 52),
 ('поддержка', 49),
 ('деньга', 47),
 ('всё', 47)]

In [50]:
bad_words = []
for k, v in bad_c.items():
    if v >= 3:
        bad_words.append(k)

In [51]:
len(bad_words)

496

In [52]:
good_c = Counter(good_corpus.split(' '))

In [53]:
good_c.most_common(20)

[('', 3723),
 ('в', 627),
 ('и', 563),
 ('я', 550),
 ('не', 370),
 ('карта', 337),
 ('что', 333),
 ('на', 288),
 ('с', 243),
 ('банк', 219),
 ('быть', 205),
 ('но', 167),
 ('всё', 164),
 ('это', 163),
 ('бот', 139),
 ('по', 137),
 ('как', 133),
 ('у', 120),
 ('поддержка', 119),
 ('за', 118)]

In [55]:
good_words = []
for k, v in good_c.items():
    if v >= 3:
        good_words.append(k)

In [57]:
len(good_words)

954

In [58]:
good_words = set(good_words)

In [59]:
bad_words = set(bad_words)

In [64]:
uniq_good = good_words - bad_words
uniq_bad = bad_words - good_words

In [68]:
uniq_good, uniq_bad

({'13',
  '14',
  '15',
  '18',
  '25',
  '300',
  '500',
  '60',
  '7',
  'bank',
  'talk',
  'viber',
  '«',
  '»',
  'автоматически',
  'адрес',
  'активно',
  'активный',
  'анализ',
  'банковский',
  'бездушный',
  'безопасность',
  'бесконтактный',
  'бесплатно',
  'бесплатный',
  'билет',
  'благо',
  'благодарить',
  'благодарность',
  'благодаря',
  'близкий',
  'блэк',
  'больший',
  'бояться',
  'брат',
  'брать',
  'будущее',
  'буквально',
  'бывать',
  'быстро',
  'быстрый',
  'бюджет',
  'важно',
  'вайбер',
  'вариант',
  'ввести',
  'везде',
  'вести',
  'вещь',
  'взгляд',
  'взять',
  'вид',
  'видимо',
  'видно',
  'виртуальный',
  'вместе',
  'вместо',
  'внимательно',
  'возвращать',
  'возникать',
  'возникнуть',
  'восстановление',
  'всегда',
  'вспомнить',
  'встречать',
  'всякий',
  'втб',
  'выбирать',
  'выбор',
  'выбрать',
  'выгодный',
  'выдать',
  'выйти',
  'выпуск',
  'выразить',
  'выслать',
  'высокий',
  'выставить',
  'выше',
  'выясниться',
  '

Создам функцию которая проверяет плохой или хорощий отзыв, отзыв вставляем ручками

In [82]:
def good_or_bad(feedback):
    feedback = feedback.lower()
    feedback = nltk.word_tokenize(feedback)
    feedback_out = []
    for fb in feedback:
        f = morph.parse(fb)[0]
        feedback_out.append(f.normal_form)
    feedback_out = set(feedback_out)
    if feedback_out & uniq_good > feedback_out & uniq_bad:
        print('Good')
    else:
        print('Bad')



In [85]:
review_good_example = '''Всем добрый день. Хочется поделиться одной ситуацией, в которой сотрудники Толкбанка помогли быстро найти решение. У меня карта блэк, все устраивает и даже очень)) Так вышло, что занимала деньги у подруги. Решила ей переслать долг обратно 11 мая по номеру телефона. СМС со ссылкой на получение улетело куда-то не туда(( Я ошиблась всего на одну цифру. Подруга ждет, я немного на нервах.

В общем, СМС тот другой человек пока не прочитал, видимо, списаний не было. Написала в чат банка, думаю, ну бот с этим не справится пока что. Вообще, конечно, его здорово развивают, но не до такой степени пока. В общем, мне ответил оператор из поддержки, живой человек)) Предложил несколько вариантов выхода из ситуации. Я выбрала оптимальный: переслать ссылку подруге, чтобы она ввела данные своей карты. Так она смогла сразу забрать нужные деньги!!! Очень благодарю за это банк!!'''
good_or_bad(review_good_example)

Good


In [87]:
review_bad_example = '''Пользуюсь картой пару лет. Купился на мессенджеры и кэшбэк.
Хоть компания и заявляет, что кэшбэк - это приятный бонус, взял карту именно ради него. На поверку оказалось, что открывается период компенсации только при хитрых условиях, легко аннулируется, и даже если он у вас открыт, то компенсировать можно не все, и не в полном объеме. 

А когда я вот уже вроде бы я вошел "в режим", и уже начал класть необходимую сумму для поддержания программы лояльности, вышло некое обновление тарифов, сделавшее пользование картой окончательно бессмысленным. В итоге 16 00 бонусов сгорели. Спасибо. За всю историю удалось снять пару раз по паре тысяч. Мне сейчас представитель банка напишет, что я сам дурак, и вообще я должен быть им благодарен за этот приятный бонус. Но если задумаетесь над приобретением карты, имейте в виду мой опыт.
Помимо этого, зависли средства. Рассчитываться ими неудобно (истерлась карта, да и программы лояльности других банков оказались прозрачнее и выгоднее), а переводы и снятие наличных теперь имеют лимит = 0, и за любую из этих операций снимают комиссию. 
Причем, даже в банкоматах "домашнего" банка (у меня ТКБ). Ну, чтож.. Сам дурак. Пусть комиссия. Пошел снимать. Угадаете сколько составила комиссия? Более 7,5%. 

Начал разбираться с банком. Бот на вопрос об ошибке снятия комиссии пичкал меня рекламой лимитов. По телефону они не консультируют, ссылаясь на заботу о клиентах (что звучало форменной издевкой). Через 15 минут переписки с ботом удалось отправить сообщение оператору. Был отправлен ждать ответа.
Дождался. Мне сообщили, что с меня должна быть снята комиссия, и вообще меня предупреждали (в таком же стиле тут отвечает представитель банка). Я задал вопрос - а почему в таком размере? Был отправлен ждать ответа. 

Ответа ждал сутки. Написали, что у меня какой-то сложный вопрос, и нужно время. И вы ведь понимаете, что все это вперемешку с общением с туповатым шаблонным ботом, вопрошающим, что именно мне не понравилось. При этом мои ответы на этот вопрос или никто не читал, или они вообще не передавались (несмотря на фразу - я передам комментарий команде). Поэтому для уточняющих вопросов приходилось формировать новое обращение.
Дождался ответа. Мне радостно сообщили, что ошиблись, и возмещают мне неверно удержанную комиссию (с восклицательным знаком). При этом баланс не изменился. Вопрос боту повис в воздухе. Когда я понял, что мне не ответят, написал еще одно обращение. Спустя еще каких-то 2-3 часа мне сообщили, что возврат комиссии идет 3 дня. 
Вот так спустя трое суток я узнал, что мне, возможно, что-то компенсируют, и что сами они про свой косяк не в курсе.

Но на самом деле, это все эмоции последних дней. А ранее была масса ситуаций, когда нужно было воспользоваться картой, а у них не работает сервис пополнения. Или шлешь запрос на операцию, а в ответ тишина. И даже неизвестно, когда отвиснет, и отвиснет ли.

Крайне ненадежный, неудобный, и непрозрачный сервис. Рекомендовать не буду. Можно простить огрехи технические при человеческом обслуживании. Можно простить черствость при четком сервисе, и выгодных условиях. Но когда ни того ни другого - вообще не вижу смысла даже связываться.'''
good_or_bad(review_bad_example)

Bad


Как видно функция раьотает, аккураси мне считать лень

# Как улучшить программу 
1 Сделать телеграм бота который будет определять хороший или плохой отзыв

2 Посчитать акураси для функции 

3 Увеличить выборку 
